In [1]:
# default_exp core

# MatchScorer
> This notebook tests some of the functionality of the `MatchScorer` class.

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# export
import random
import math


class Scorer:
    def __init__(self, home_score=0, away_score=0, score_diff=1):
        # init with score or start with 0-0
        self.home_score = home_score
        self.away_score = away_score
        self.score_diff = score_diff

    def increase(self, is_home, score_diff=None):
        if not self.finished:
            score_diff = score_diff if score_diff else self.score_diff
            if is_home:
                self.home_score += score_diff
            else:
                self.away_score += score_diff

    def decrease(self, is_home, score_diff=None):
        if not self.finished:
            score_diff = score_diff if score_diff else self.score_diff
            if is_home:
                self.home_score -= score_diff
            else:
                self.away_score -= score_diff

    def update(self, home_score, away_score):
        self.home_score = home_score
        self.away_score = away_score

    def __repr__(self):
        return f"{self.__class__.__name__}({self.home_score}, {self.away_score})"

    @property
    def finished(self):
        raise NotImplementedError('You need to implement this function in your Scorer')

    @property
    def is_winner_home(self):
        """returns winner:
           - still playing -> None
           - True: home won
           - False: away won
        """
        if self.finished:
            return self.home_score > self.away_score
        return None

    def get_score(self):
        return f"{self.home_score}:{self.away_score}"



class TiebreakScorer(Scorer):
    final_point = 7
    min_distance = 2

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    @property
    def finished(self):
        # someone reaches seven points and is at least two points ahead
        if ((self.home_score >= self.final_point or self.away_score >= self.final_point)
           and abs(self.home_score - self.away_score) >= self.min_distance):
            return True

        return False

class MatchTiebreakScorer(Scorer):
    final_point = 10
    min_distance = 2

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    @property
    def finished(self):
        # someone reaches seven points and is at least two points ahead
        if ((self.home_score >= self.final_point or self.away_score >= self.final_point)
           and abs(self.home_score - self.away_score) >= self.min_distance):
            return True
        return False


# Need: Track winner, track stats

class GamePointScorer(Scorer):
    internal_to_game_mapping = {0: '0', 1:'15', 2:'30', 3:'40', 4:'Ad', 5:'F'}
    game_to_internal_mapping = {'0':0, '15':1, '30':2, '40':3, 'Ad':4, 'F':5}
    final_point = 4
    
    def __init__(self, home_score='0', away_score='0', score_diff=1, deciding_point=False):
        # init with score or start with 0-0
        int_home_score = self.game_to_internal_mapping[home_score]
        int_away_score = self.game_to_internal_mapping[away_score]
        self.deciding_point = deciding_point
        super().__init__(int_home_score, int_away_score, score_diff)
        
    def increase(self, is_home, score_diff=None):
        super().increase(is_home, score_diff)                
        if self.home_score == 4 and self.away_score == 4:
            self.home_score, self.away_score = 3, 3
            
    def update(self, home_score, away_score):
        try:
            self.home_score = self.game_to_internal_mapping[home_score]
            self.away_score = self.game_to_internal_mapping[away_score]
        except:
            raise ValueError('No valid score ({list(self.internal_to_game_mapping.values())})')
                      
    @property
    def finished(self):
        # someone reaches Ad point and is at least two points ahead
        max_score = max(self.home_score, self.away_score)
        min_score = min(self.home_score, self.away_score)
        diff_score = max_score - min_score
        
        if max_score >= 4:
            if diff_score>=2:
                return True
        if self.deciding_point:
            if max_score == 4 and min_score == 3:
                return True
            
        return False
    
    def __repr__(self):
        home_score = self.internal_to_game_mapping[self.home_score]
        away_score = self.internal_to_game_mapping[self.away_score]
        return f"{self.__class__.__name__}({home_score}, {away_score})"
    
    def get_score(self):
        home_score = self.internal_to_game_mapping[self.home_score]
        away_score = self.internal_to_game_mapping[self.away_score]
        return f"{home_score}:{away_score}"


# Need to track winner of each GamePoint to decide when it is finished (return True/False should be enough)
class SetTracker(Scorer):
    min_distance = 2
    tiebreak = True # 6-6 -> Tiebreak

    def __init__(self, home_score=0, away_score=0, score_diff=1, final_point=6, deciding_point=False):
        super().__init__(home_score,away_score,score_diff)
        self.games = [GamePointScorer(deciding_point=deciding_point) for i in range(final_point*2)]
        self.games = self.games + [TiebreakScorer()]
        self.current_gamepoint = home_score + away_score
        self.final_point = final_point
        self.deciding_point = deciding_point
        #self.score_diff = score_diff
        #self.home_score = 0
        #self.away_score = 0
        # points[12] of PointTracker(deuce=True)
        # tiebreak of TiebreakTracker

    @property
    def is_winner_home(self):
        """returns winner:
           - still playing -> None
           - True: home won
           - False: away won
        """
        if self.finished:
            return self.home_score > self.away_score
        return None

    @property
    def current_game(self):
        return self.games[self.current_gamepoint]

    @property
    def finished(self):
        # someone reaches seven points and is at least two points ahead
        max_score = max(self.home_score, self.away_score)
        min_score = min(self.home_score, self.away_score)
        diff_score = max_score - min_score
        if max_score == self.final_point + 1:
            return True
        elif ((self.home_score == self.final_point or self.away_score == self.final_point)
            and diff_score>=2):
            return True

        return False

    def increase(self, is_home, score_diff=None):
        if not self.finished:
            # score_diff = score_diff if score_diff else self.score_diff
            self.current_game.increase(is_home, score_diff)
            if self.current_game.finished:
                if self.current_game.is_winner_home is not None:
                    if self.current_game.is_winner_home:
                        self.home_score+=1
                    else:
                        self.away_score+=1
                if self.current_gamepoint < len(self.games)-1:
                    self.current_gamepoint += 1


    def update(self, home_score, away_score):
        self.home_score = home_score
        self.away_score = away_score
        self.current_gamepoint = home_score + away_score

    def __repr__(self):
        return f"{self.__class__.__name__}({self.home_score}, {self.away_score}), {self.current_game}"

    def get_score(self):
        return f"{self.home_score}:{self.away_score}"


class MatchTracker(Scorer):

    def __init__(self, home_score=0, away_score=0, score_diff=1, best_of=3, match_tiebreak=True, deciding_point=False):
        super().__init__(home_score,away_score,score_diff)
        self.best_of = best_of
        self.match_tiebreak = match_tiebreak
        self.sets = [SetTracker(deciding_point=deciding_point) for i in range(best_of-1)]
        if self.match_tiebreak:
            self.sets = self.sets + [MatchTiebreakScorer()]# might add MatchTiebreak here
        else:
            self.sets = self.sets + [SetTracker(deciding_point=deciding_point)]
        self.current_set_id = home_score + away_score

    @property
    def is_winner_home(self):
        """returns winner:
           - still playing -> None
           - True: home won
           - False: away won
        """
        if self.finished:
            return self.home_score > self.away_score
        return None

    @property
    def current_set(self):
        return self.sets[self.current_set_id]

    @property
    def finished(self):
        # someone reaches seven points and is at least two points ahead
        max_score = max(self.home_score, self.away_score)
        min_score = min(self.home_score, self.away_score)
        if max_score == math.ceil(self.best_of/2):
            return True
        return False

    def increase(self, is_home, score_diff=None):
        if not self.finished:
            # score_diff = score_diff if score_diff else self.score_diff
            self.current_set.increase(is_home, score_diff)
            if self.current_set.finished:
                if self.current_set.is_winner_home is not None:
                    if self.current_set.is_winner_home:
                        self.home_score+=1
                    else:
                        self.away_score+=1
                if (self.current_set_id < len(self.sets)-1) and not self.finished:
                    self.current_set_id += 1


    def update(self, home_score, away_score):
        self.home_score = home_score
        self.away_score = away_score
        self.current_set_id = home_score + away_score

    def __repr__(self):
        return f"{self.__class__.__name__}({self.home_score}, {self.away_score}), {self.current_set}"

    def get_score(self):
        finished_sets = [set_.get_score() for i, set_ in enumerate(self.sets) if i <= self.current_set_id]
        return ";".join(finished_sets) + '-' + self.current_set.current_game.get_score()

    @classmethod
    def WIMBLEDON(cls):
        BEST_OF=5
        MATCH_TIEBREAK=False
        FINAL_POINT=12

        instance = cls(
            match_tiebreak=MATCH_TIEBREAK,
            best_of=BEST_OF)

        instance.sets = [SetTracker() for i in range(BEST_OF-1)]
        instance.sets = instance.sets + [SetTracker(final_point=FINAL_POINT)]

        return instance


# class MatchTrackerWimbledon(MatchTracker):
#     BEST_OF=5
#     MATCH_TIEBREAK=False
#     FINAL_POINT=12

#     def __init__(self, **kwargs):
#         super().__init__(
#             match_tiebreak=self.MATCH_TIEBREAK,
#             best_of=self.BEST_OF,
#             **kwargs)

#         self.sets = [SetTracker() for i in range(self.BEST_OF-1)]
#         self.sets = self.sets + [SetTracker(final_point=self.FINAL_POINT)]

In [4]:
scores = random.choices(population=[True, False], k=20)

In [5]:
tbscorer = TiebreakScorer()

for score in scores:
    tbscorer.increase(score)
    print(tbscorer)
    if tbscorer.finished:
        print('Game is over.')
        print(tbscorer.is_winner_home)
        break

TiebreakScorer(0, 1)
TiebreakScorer(1, 1)
TiebreakScorer(1, 2)
TiebreakScorer(1, 3)
TiebreakScorer(1, 4)
TiebreakScorer(2, 4)
TiebreakScorer(2, 5)
TiebreakScorer(3, 5)
TiebreakScorer(4, 5)
TiebreakScorer(4, 6)
TiebreakScorer(5, 6)
TiebreakScorer(6, 6)
TiebreakScorer(6, 7)
TiebreakScorer(7, 7)
TiebreakScorer(7, 8)
TiebreakScorer(8, 8)
TiebreakScorer(9, 8)
TiebreakScorer(9, 9)
TiebreakScorer(10, 9)
TiebreakScorer(10, 10)


In [6]:
tbscorer = MatchTiebreakScorer()

for score in scores:
    tbscorer.increase(score)
    print(tbscorer)
    if tbscorer.finished:
        print('Game is over.')
        print(tbscorer.is_winner_home)
        break

MatchTiebreakScorer(0, 1)
MatchTiebreakScorer(1, 1)
MatchTiebreakScorer(1, 2)
MatchTiebreakScorer(1, 3)
MatchTiebreakScorer(1, 4)
MatchTiebreakScorer(2, 4)
MatchTiebreakScorer(2, 5)
MatchTiebreakScorer(3, 5)
MatchTiebreakScorer(4, 5)
MatchTiebreakScorer(4, 6)
MatchTiebreakScorer(5, 6)
MatchTiebreakScorer(6, 6)
MatchTiebreakScorer(6, 7)
MatchTiebreakScorer(7, 7)
MatchTiebreakScorer(7, 8)
MatchTiebreakScorer(8, 8)
MatchTiebreakScorer(9, 8)
MatchTiebreakScorer(9, 9)
MatchTiebreakScorer(10, 9)
MatchTiebreakScorer(10, 10)


In [7]:
game_scorer = GamePointScorer()

for score in scores:
    game_scorer.increase(score)
    print(game_scorer)
    if game_scorer.finished:
        print('Game is over.')
        print(game_scorer.is_winner_home)
        break

GamePointScorer(0, 15)
GamePointScorer(15, 15)
GamePointScorer(15, 30)
GamePointScorer(15, 40)
GamePointScorer(15, Ad)
Game is over.
False


In [8]:
game_scorer = GamePointScorer('40','30')
scores = [False, False, True, True, False, False, False]
for score in scores:
    game_scorer.increase(score)
    print(game_scorer)
    if game_scorer.finished:
        print('Game is over.')
        print(game_scorer.is_winner_home)
        break

GamePointScorer(40, 40)
GamePointScorer(40, Ad)
GamePointScorer(40, 40)
GamePointScorer(Ad, 40)
GamePointScorer(40, 40)
GamePointScorer(40, Ad)
GamePointScorer(40, F)
Game is over.
False


In [9]:
set_scorer = SetTracker(6,6)
scores = random.choices(population=[True, False], k=36)

for score in scores:
    set_scorer.increase(score)
    print(set_scorer)
    if set_scorer.finished:
        print('Game is over.')
        print(set_scorer.is_winner_home)
        break

SetTracker(6, 6), TiebreakScorer(0, 1)
SetTracker(6, 6), TiebreakScorer(1, 1)
SetTracker(6, 6), TiebreakScorer(1, 2)
SetTracker(6, 6), TiebreakScorer(1, 3)
SetTracker(6, 6), TiebreakScorer(2, 3)
SetTracker(6, 6), TiebreakScorer(2, 4)
SetTracker(6, 6), TiebreakScorer(3, 4)
SetTracker(6, 6), TiebreakScorer(4, 4)
SetTracker(6, 6), TiebreakScorer(5, 4)
SetTracker(6, 6), TiebreakScorer(6, 4)
SetTracker(6, 6), TiebreakScorer(6, 5)
SetTracker(7, 6), TiebreakScorer(7, 5)
Game is over.
True


In [10]:
set_scorer = SetTracker(3,3, deciding_point=True)
scores = random.choices(population=[True, False], k=36)

for score in scores:
    set_scorer.increase(score)
    print(set_scorer)
    if set_scorer.finished:
        print('Game is over.')
        print(set_scorer.is_winner_home)
        break

SetTracker(3, 3), GamePointScorer(0, 15)
SetTracker(3, 3), GamePointScorer(15, 15)
SetTracker(3, 3), GamePointScorer(15, 30)
SetTracker(3, 3), GamePointScorer(15, 40)
SetTracker(3, 3), GamePointScorer(30, 40)
SetTracker(3, 3), GamePointScorer(40, 40)
SetTracker(4, 3), GamePointScorer(0, 0)
SetTracker(4, 3), GamePointScorer(0, 15)
SetTracker(4, 3), GamePointScorer(0, 30)
SetTracker(4, 3), GamePointScorer(15, 30)
SetTracker(4, 3), GamePointScorer(15, 40)
SetTracker(4, 3), GamePointScorer(30, 40)
SetTracker(4, 3), GamePointScorer(40, 40)
SetTracker(5, 3), GamePointScorer(0, 0)
SetTracker(5, 3), GamePointScorer(0, 15)
SetTracker(5, 3), GamePointScorer(0, 30)
SetTracker(5, 3), GamePointScorer(0, 40)
SetTracker(5, 4), GamePointScorer(0, 0)
SetTracker(5, 4), GamePointScorer(0, 15)
SetTracker(5, 4), GamePointScorer(0, 30)
SetTracker(5, 4), GamePointScorer(15, 30)
SetTracker(5, 4), GamePointScorer(30, 30)
SetTracker(5, 4), GamePointScorer(30, 40)
SetTracker(5, 5), GamePointScorer(0, 0)
SetTrack

In [11]:
match_scorer = MatchTracker(0,1, match_tiebreak=False)
scores = random.choices(population=[True, False], k=36)

for score in scores:
    match_scorer.increase(score)
    print(match_scorer.get_score())
    if match_scorer.finished:
        print('Match is over.')
        print(match_scorer.is_winner_home)
        break

0:0;0:0-15:0
0:0;0:0-30:0
0:0;0:0-40:0
0:0;0:0-40:15
0:0;0:0-40:30
0:0;1:0-0:0
0:0;1:0-0:15
0:0;1:0-0:30
0:0;1:0-0:40
0:0;1:0-15:40
0:0;1:1-0:0
0:0;1:1-15:0
0:0;1:1-15:15
0:0;1:1-30:15
0:0;1:1-40:15
0:0;2:1-0:0
0:0;2:1-15:0
0:0;2:1-30:0
0:0;2:1-40:0
0:0;2:1-40:15
0:0;3:1-0:0
0:0;3:1-0:15
0:0;3:1-15:15
0:0;3:1-30:15
0:0;3:1-40:15
0:0;3:1-40:30
0:0;4:1-0:0
0:0;4:1-0:15
0:0;4:1-0:30
0:0;4:1-0:40
0:0;4:2-0:0
0:0;4:2-15:0
0:0;4:2-30:0
0:0;4:2-40:0
0:0;4:2-40:15
0:0;5:2-0:0


In [12]:
match_scorer = MatchTracker(0,0, match_tiebreak=False)
scores = random.choices(population=[True, False], k=200)

for score in scores:
    match_scorer.increase(score)
    print(match_scorer.get_score())
    if match_scorer.finished:
        print('Match is over.')
        print(match_scorer.is_winner_home)
        break

0:0-0:15
0:0-0:30
0:0-0:40
0:1-0:0
0:1-0:15
0:1-0:30
0:1-15:30
0:1-15:40
0:1-30:40
0:1-40:40
0:1-Ad:40
0:1-40:40
0:1-Ad:40
0:1-40:40
0:1-40:Ad
0:2-0:0
0:2-0:15
0:2-0:30
0:2-15:30
0:2-30:30
0:2-40:30
0:2-40:40
0:2-40:Ad
0:3-0:0
0:3-15:0
0:3-30:0
0:3-40:0
0:3-40:15
0:3-40:30
0:3-40:40
0:3-40:Ad
0:4-0:0
0:4-15:0
0:4-30:0
0:4-30:15
0:4-30:30
0:4-40:30
0:4-40:40
0:4-40:Ad
0:4-40:40
0:4-40:Ad
0:5-0:0
0:5-0:15
0:5-0:30
0:5-15:30
0:5-30:30
0:5-40:30
0:5-40:40
0:5-Ad:40
1:5-0:0
1:5-0:15
1:5-0:30
1:5-0:40
1:5-15:40
1:5-30:40
1:6;0:0-0:0
1:6;0:0-0:15
1:6;0:0-15:15
1:6;0:0-15:30
1:6;0:0-30:30
1:6;0:0-40:30
1:6;1:0-0:0
1:6;1:0-0:15
1:6;1:0-15:15
1:6;1:0-30:15
1:6;1:0-40:15
1:6;1:0-40:30
1:6;1:0-40:40
1:6;1:0-Ad:40
1:6;1:0-40:40
1:6;1:0-Ad:40
1:6;1:0-40:40
1:6;1:0-40:Ad
1:6;1:0-40:40
1:6;1:0-Ad:40
1:6;2:0-0:0
1:6;2:0-0:15
1:6;2:0-0:30
1:6;2:0-15:30
1:6;2:0-30:30
1:6;2:0-30:40
1:6;2:1-0:0
1:6;2:1-0:15
1:6;2:1-15:15
1:6;2:1-30:15
1:6;2:1-40:15
1:6;3:1-0:0
1:6;3:1-15:0
1:6;3:1-30:0
1:6;3:1-40:0
1:6;4:1

In [13]:
wimbledon_match_scorer = MatchTracker.WIMBLEDON()

scores = random.choices(population=[True, False], k=400)

for score in scores:
    wimbledon_match_scorer.increase(score)
    print(wimbledon_match_scorer.get_score())
    if wimbledon_match_scorer.finished:
        print('Match is over.')
        print(wimbledon_match_scorer.is_winner_home)
        break

0:0-15:0
0:0-15:15
0:0-15:30
0:0-30:30
0:0-30:40
0:0-40:40
0:0-40:Ad
0:0-40:40
0:0-40:Ad
0:0-40:40
0:0-Ad:40
0:0-40:40
0:0-40:Ad
0:1-0:0
0:1-0:15
0:1-15:15
0:1-30:15
0:1-30:30
0:1-40:30
1:1-0:0
1:1-0:15
1:1-15:15
1:1-30:15
1:1-40:15
2:1-0:0
2:1-0:15
2:1-15:15
2:1-30:15
2:1-40:15
2:1-40:30
2:1-40:40
2:1-Ad:40
2:1-40:40
2:1-40:Ad
2:2-0:0
2:2-15:0
2:2-15:15
2:2-30:15
2:2-30:30
2:2-40:30
3:2-0:0
3:2-15:0
3:2-30:0
3:2-30:15
3:2-40:15
4:2-0:0
4:2-0:15
4:2-0:30
4:2-0:40
4:3-0:0
4:3-0:15
4:3-0:30
4:3-0:40
4:3-15:40
4:3-30:40
4:3-40:40
4:3-Ad:40
5:3-0:0
5:3-0:15
5:3-15:15
5:3-15:30
5:3-30:30
5:3-30:40
5:4-0:0
5:4-15:0
5:4-15:15
5:4-30:15
5:4-40:15
6:4;0:0-0:0
6:4;0:0-0:15
6:4;0:0-0:30
6:4;0:0-15:30
6:4;0:0-15:40
6:4;0:0-30:40
6:4;0:0-40:40
6:4;0:0-40:Ad
6:4;0:0-40:40
6:4;0:0-Ad:40
6:4;1:0-0:0
6:4;1:0-15:0
6:4;1:0-30:0
6:4;1:0-30:15
6:4;1:0-40:15
6:4;1:0-40:30
6:4;2:0-0:0
6:4;2:0-0:15
6:4;2:0-0:30
6:4;2:0-0:40
6:4;2:1-0:0
6:4;2:1-15:0
6:4;2:1-30:0
6:4;2:1-30:15
6:4;2:1-30:30
6:4;2:1-40:30
6:4;2:

In [14]:
final_set = SetTracker(12, 12, final_point=12)

for score in scores:
    final_set.increase(score)
    print(final_set)
    if final_set.finished:
        print('Game is over.')
        print(final_set.is_winner_home)
        break

SetTracker(12, 12), TiebreakScorer(1, 0)
SetTracker(12, 12), TiebreakScorer(1, 1)
SetTracker(12, 12), TiebreakScorer(1, 2)
SetTracker(12, 12), TiebreakScorer(2, 2)
SetTracker(12, 12), TiebreakScorer(2, 3)
SetTracker(12, 12), TiebreakScorer(3, 3)
SetTracker(12, 12), TiebreakScorer(3, 4)
SetTracker(12, 12), TiebreakScorer(4, 4)
SetTracker(12, 12), TiebreakScorer(4, 5)
SetTracker(12, 12), TiebreakScorer(5, 5)
SetTracker(12, 12), TiebreakScorer(6, 5)
SetTracker(12, 12), TiebreakScorer(6, 6)
SetTracker(12, 12), TiebreakScorer(6, 7)
SetTracker(12, 13), TiebreakScorer(6, 8)
Game is over.
False


In [ ]:
from nbdev.export import notebook2script; notebook2script()